In [3]:
import requests
from bs4 import BeautifulSoup
import re
import warnings
import json
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

In [ ]:
path = 'https://www.imdb.com/chart/top/?ref_=nv_mv_250'
base = 'https://www.imdb.com'
headers = {'User-Agent': 'Safari/605.1.15'}

# GET LINKs

In [3]:
r = requests.get(f'{path}')
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('tbody', class_ = 'lister-list')
movies = s.find_all('td',class_ = 'titleColumn')
links = []
for movie in movies:
    link = movie.a['href']
    links.append(f'{link}')

save links

In [4]:
links = np.array(links)
np.save('top_250_Linkes',links)

load links

In [10]:
links = np.load('top_250_Linkes.npy')

# GET DATA

In [ ]:
xrror_gross_us_canda = []
xrror_runtime = []
xrror_genre_movie = []
xrror_cast = []
xrror_cast = []
xrror_storyline = []
xrror = []

movies = pd.DataFrame(columns = ['id','title','year','runtime','parental_guide','gross_us_canda'])
genre_movie =  pd.DataFrame(columns = ['movie_id','genre'])
cast = pd.DataFrame(columns = ['movie_id','person_id'])
crew = pd.DataFrame(columns = ['movie_id','person_id','role'])
person = pd.DataFrame(columns = ['person_id','name'])
storyline = pd.DataFrame(columns = ['movie_id','content'])

In [129]:
for link in links:
    
    r = requests.get(f'{base}{link}', headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')

    try:
        movie_id = link[9:-1]
        try:
            content = soup.find('div', class_ = 'ipc-html-content-inner-div').text
            storyline.loc[len(storyline.index)] = [movie_id,content]

        except Exception as e:
            storyline.loc[len(storyline.index)] = [movie_id,'']
            print(f'storyline {e}')
            xrror_storyline.append(link)
        title = soup.find('span', class_ = 'sc-afe43def-1 fDTGTb').text
        info = soup.find_all('li',role ='presentation',class_="ipc-inline-list__item")
        year = int(info[3].text)
        parental_guide = info[4].text
        runtime = info[5].text
        try :
            runtime = int(runtime[0])*60 + int(runtime[3:-1])
        except Exception as e: 
            print(f'runtime {e}')       
            xrror_runtime.append(link)
        try:
            gross_us_canda = soup.find_all('span', class_ = 'ipc-metadata-list-item__list-content-item')[3].text
        except Exception as e:
            gross_us_canda = ''
            print(f'gross_us_canda {e}')       
            xrror_gross_us_canda.append(link)
        movies.loc[len(movies.index)] = [movie_id,title,year,runtime,parental_guide,gross_us_canda]
        genres = soup.find_all('a', class_ = 'ipc-chip ipc-chip--on-baseAlt')
        for g in genres:
            try: 
                genre_movie.loc[len(genre_movie.index)] = [movie_id,g.text]
            except Exception as e: 
                print(f'genre_movie {e}')       
                xrror_genre_movie.append(link)
        for i in soup.find_all('li', 
            class_ = 'ipc-metadata-list__item')[:3]:
            span = i.find_all('a'
                ,class_ = 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
            role = f'{i.text[:8]}'
            for s in span:
                name = s.text
                p_link = s['href']
                person.loc[len(person.index)] = [p_link[8:-15].zfill(7),name]
                person.drop_duplicates(inplace = True)
                if (role[:5] == 'Stars'):
                    try:
                        cast.loc[len(cast.index)] = [movie_id,p_link[8:-15].zfill(7)]
                    except Exception as e: 
                        print(f'cast {e}')       
                        xrror_cast.append(link)
                else:
                    try:
                        crew.loc[len(crew.index)] = [movie_id,p_link[8:-15].zfill(7),role[:-2]]
                    except Exception as e: 
                        print(f'crew {e}')       
                        xrror_crew.append(link)
                        
    except Exception as e: 
        print(e)       
        xrror.append(link)

Handle Error HARDCODE :|

In [ ]:
runtime = movies[movies['id'] == '0082096'].parental_guide.iloc[0]
false_value = movies[movies['id'] == '0082096'].runtime.iloc[0]
movies['parental_guide'].replace(runtime,'',inplace = True)
movies['runtime'].replace(false_value, int(runtime[0])*60 + int(runtime[3:-1]),inplace = True)

movies['runtime'].replace('2h',120,inplace = True)
movies['runtime'].replace('3h',180,inplace = True)
movies['runtime'].replace('4h',240,inplace = True)

save links data 

In [132]:
cast.to_csv('cast.csv',index=False)
crew.to_csv('crew.csv',index=False)
person.to_csv('person.csv',index=False)
genre_movie.to_csv('genre_movie.csv',index=False)
movies.to_csv('movies.csv',index=False)
storyline.to_csv('storyline.csv',index=False)

# LOAD CSVs 

In [42]:
movies = pd.read_csv('movies.csv',dtype=object)
genre_movie =  pd.read_csv('genre_movie.csv',dtype=object)
cast = pd.read_csv('cast.csv',dtype=object)
crew = pd.read_csv('crew.csv',dtype=object)
person = pd.read_csv('person.csv',dtype=object)
storyline = pd.read_csv('storyline.csv',dtype=object)

Fix Writers

In [22]:
crew.loc[crew['role'].str.startswith('Writer', na=False) , 'role'] = 'Writer'
crew.to_csv('crew.csv',index=False)